In [14]:
import psycopg2
import pandas as pd

# Database connection parameters
DB_HOST = "localhost"
DB_NAME = "todoler"
DB_USER = "dinesh"
DB_PASSWORD = "neverworry"
DB_PORT = "5432"  # Default PostgreSQL port is 5432

def execute_query(query : str):
    
    # SQL query to execute
    
    try:
        # Establish the connection
        connection = psycopg2.connect(
            host=DB_HOST,
            database=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            port=DB_PORT
        )
    
        pd.set_option('display.max_rows', None)
    
        # Load data into a DataFrame
        df = pd.read_sql_query(query, connection)
        return df
    
    except Exception as error:
        print(f"Error: {error}")
    
    finally:
        # Close the connection
        if connection:
            connection.close()

In [15]:
query = "SELECT * FROM tasks;"
execute_query(query)

/tmp/ipykernel_21285/3128263836.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,id,title,subtasks,due_date,comments,description,task_status,created_time,project_id
0,146,request from tests,"[subtask1, subtask2]",2024-02-04,"[this is the first test comment, this is the s...",this is test description,in progress,2024-06-20 06:01:54.049761,1
1,116,request from tests,"[subtask1, subtask2]",2024-02-04,"[this is the first test comment, this is the s...",this is test description,in progress,2024-06-02 13:06:46.252726,1
2,120,request from tests,"[subtask1, subtask2]",2024-02-04,"[this is the first test comment, this is the s...",this is test description,in progress,2024-06-02 13:06:50.489150,1
3,124,request from tests,"[subtask1, subtask2]",2024-02-04,"[this is the first test comment, this is the s...",this is test description,in progress,2024-06-05 07:20:29.895496,1
4,114,trigger test on 114,"[subtask1, subtask2]",2024-02-04,"[let us try with thired change, this is the se...",this is test description for triggger test,completed,2024-05-26 06:13:20.708379,1
5,128,request from tests,"[subtask1, subtask2]",2024-02-04,"[this is the first test comment, this is the s...",this is test description,in progress,2024-06-07 08:04:25.715067,1
6,132,request from tests,"[subtask1, subtask2]",2024-02-04,"[this is the first test comment, this is the s...",this is test description,in progress,2024-06-08 09:40:07.089451,1
7,136,request from tests,"[subtask1, subtask2]",2024-02-04,"[this is the first test comment, this is the s...",this is test description,in progress,2024-06-09 06:53:18.731997,1
8,142,request from tests,"[subtask1, subtask2]",2024-02-04,"[this is the first test comment, this is the s...",this is test description,in progress,2024-06-15 20:29:30.289713,1
9,115,this is a good day,"[identify root cause sdfsdfdsf reported bugs, ...",2024-04-20,"[make task as pending task, coordinate with qa...",addressing reported bugs in the application,pending,2024-05-26 06:13:56.382136,1


In [16]:
query = """
SELECT * FROM task_status_history
ORDER BY task_id, update_time DESC
"""

execute_query(query)

/tmp/ipykernel_21285/3128263836.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,id,task_id,old_status,new_status,update_time
0,16,93,completed,to do,2024-06-16 14:14:15.144808
1,4,93,to do,completed,2024-06-03 09:05:37.789011
2,3,93,completed,to do,2024-06-03 08:57:46.912973
3,5,114,pending,completed,2024-06-05 07:37:26.251455
4,13,122,in progress,to do,2024-06-11 08:25:16.634784
5,10,130,to do,in progress,2024-06-11 07:14:02.049933
6,9,130,in progress,to do,2024-06-11 07:13:49.333241


In [17]:
query = """
SELECT
    tasks.project_id,
    task_status_history.new_status,
    date_trunc('month', task_status_history.update_time) AS month,
    COUNT(*) AS status_count
FROM
    task_status_history
RIGHT JOIN
    tasks
ON
    task_status_history.task_id = tasks.id
GROUP BY
    tasks.project_id,
    task_status_history.new_status,
    month
ORDER BY
    tasks.project_id,
    month,
    task_status_history.new_status
LIMIT 20;

"""

execute_query(query)

/tmp/ipykernel_21285/3128263836.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,project_id,new_status,month,status_count
0,1,completed,2024-06-01,2
1,1,in progress,2024-06-01,1
2,1,to do,2024-06-01,4
3,1,None,NaT,40


In [18]:
query = """
SELECT
    tasks.project_id,
    task_status_history.new_status,
    task_status_history.update_time,
    tasks.id
FROM
    task_status_history
RIGHT JOIN
    tasks
ON
    task_status_history.task_id = tasks.id
ORDER BY
    tasks.id
LIMIT 200;

"""

execute_query(query)

/tmp/ipykernel_21285/3128263836.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,project_id,new_status,update_time,id
0,1,to do,2024-06-03 08:57:46.912973,93
1,1,completed,2024-06-03 09:05:37.789011,93
2,1,to do,2024-06-16 14:14:15.144808,93
3,1,None,NaT,94
4,1,None,NaT,95
5,1,None,NaT,96
6,1,None,NaT,98
7,1,None,NaT,99
8,1,None,NaT,100
9,1,None,NaT,102


In [19]:
query = """
WITH LatestStatus AS (
    SELECT
        tasks.project_id,
        task_status_history.new_status,
        task_status_history.update_time,
        tasks.id,
        ROW_NUMBER() OVER (PARTITION BY tasks.id ORDER BY task_status_history.update_time DESC) AS rn
    FROM
        task_status_history
    RIGHT JOIN
        tasks
    ON
        task_status_history.task_id = tasks.id
)
SELECT
    project_id,
    new_status,
    update_time,
    id
FROM
    LatestStatus
WHERE
    rn = 1
ORDER BY
    update_time DESC
LIMIT 200;


"""

execute_query(query)

/tmp/ipykernel_21285/3128263836.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,project_id,new_status,update_time,id
0,1,None,NaT,149
1,1,None,NaT,94
2,1,None,NaT,95
3,1,None,NaT,96
4,1,None,NaT,98
5,1,None,NaT,99
6,1,None,NaT,100
7,1,None,NaT,102
8,1,None,NaT,103
9,1,None,NaT,104


In [25]:
query = """
WITH LatestStatus AS (
    SELECT
        tasks.project_id,
        task_status_history.new_status,
        task_status_history.update_time,
        tasks.id,
        ROW_NUMBER() OVER (PARTITION BY tasks.id ORDER BY task_status_history.update_time DESC) AS rn
    FROM
        task_status_history
    RIGHT JOIN
        tasks
    ON
        task_status_history.task_id = tasks.id
),
MonthlyStatus AS (
    SELECT
        project_id,
        new_status,
        date_trunc('month', update_time) AS month,
        id
    FROM
        LatestStatus
    WHERE
        rn = 1
)
SELECT
    month,
    new_status,
    project_id,
    COUNT(*) AS status_count
FROM
    MonthlyStatus
    WHERE project_id = 1
    AND 
    new_status='completed'
GROUP BY
    project_id, month, new_status
ORDER BY
    month DESC, new_status

"""

execute_query(query)

/tmp/ipykernel_21285/3128263836.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,month,new_status,project_id,status_count
0,2024-06-01,completed,1,1
